# Castellers SQL Query Testing

This notebook provides an easy way to test SQL queries with different parameters and see the results in nice DataFrame tables.


In [1]:
# Import required libraries
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import pandas as pd
import sqlite3
from llm_sql import LLMSQLGenerator

# Set pandas display options to show all content without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Initialize the generator
generator = LLMSQLGenerator()


/Users/janreigtorra/MAC/castellers/.venv/lib/python3.13/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Test Query Function

This function tests SQL queries and displays results in a DataFrame format.


In [11]:
def test_query(query_type: str, colla = None, location = None, year = None, castell = None, diada = None, status = None, jornada = None, edition = None, limit: int = 10, debug: bool = False):
    """
    Simple function to test SQL queries with given parameters
    
    Args:
        query_type: Type of query ("millor_diada", "millor_castell", "castell_historia")
        colla: Colla name or list of colla names (optional)
        location: Location/city name or list of locations (optional)
        year: Year or list of years (optional)
        castell: Castell name or list of castell names (optional)
        diada: Diada name or list of diada names (optional)
        status: Status (optional) - Descarregat, Carregat, Intent, Intent desmuntat
        limit: Number of results to return (default 10)
    """
    print(f"\n{'='*60}")
    print(f"TESTING: {query_type.upper()}")
    print(f"{'='*60}")
    
    # Import Castell class
    from utility_functions import Castell
    
    # Build entities dictionary
    entities = {}
    if colla:
        # Handle both single values and lists
        if isinstance(colla, list):
            entities["colla"] = colla
        else:
            entities["colla"] = [colla]
        print(f"Colla: {colla}")
    if location:
        # Handle both single values and lists
        if isinstance(location, list):
            entities["llocs"] = location
        else:
            entities["llocs"] = [location]
        print(f"Location: {location}")
    if year:
        # Handle both single values and lists
        if isinstance(year, list):
            entities["anys"] = year
        else:
            entities["anys"] = [year]
        print(f"Year: {year}")
    if castell:
        # Handle castell with optional status
        if status:
            if isinstance(castell, list):
                entities["castells"] = [Castell(castell_code=c, status=status) for c in castell]
                print(f"Castell: {castell} (Status: {status})")
            else:
                entities["castells"] = [Castell(castell_code=castell, status=status)]
                print(f"Castell: {castell} (Status: {status})")
        else:
            if isinstance(castell, list):
                entities["castells"] = [Castell(castell_code=c) for c in castell]
                print(f"Castell: {castell}")
            else:
                entities["castells"] = [Castell(castell_code=castell)]
                print(f"Castell: {castell}")
    if diada:
        # Handle both single values and lists
        if isinstance(diada, list):
            entities["diades"] = diada
        else:
            entities["diades"] = [diada]
        print(f"Diada: {diada}")

    if jornada:
        # Handle both single values and lists
        if isinstance(jornada, list):
            entities["jornades"] = jornada
        else:
            entities["jornades"] = [jornada]
        print(f"Jornada: {jornada}")

    if edition:
        # Handle both single values and lists
        if isinstance(edition, list):
            entities["edicions"] = edition
        else:
            entities["edicions"] = [edition]
        print(f"Edició: {edition}")
    
    # Add limit to entities
    entities["limit"] = limit
    
    print(f"Limit: {limit}")
    print("-" * 50)
    
    try:
        # Generate SQL query
        sql_query, params = generator.create_sql_query(
            question=f"Test {query_type} query",
            entities=entities,
            sql_query_type=query_type
        )
        
        if sql_query is None:
            print("Failed to generate SQL query")
            return None
        
        # Add limit to params if not already present
        if 'limit' not in params:
            params['limit'] = limit
        
        if debug:
            print(f"Generated SQL Query:")
            print(sql_query)
            print(f"\nParameters: {params}")
        
        # Execute query
        results = generator.execute_sql_query(sql_query, params)
        
        if not results:
            print("No results found")
            return None
        
        print(f"\nResults ({len(results)} rows):")
        print("-" * 80)
        
        # Convert sqlite3.Row objects to DataFrame with proper column names
        if results:
            # Get column names from the first row
            column_names = list(results[0].keys())
            
            # Convert rows to list of dictionaries
            data = [dict(row) for row in results]
            
            # Create DataFrame with proper column names
            df = pd.DataFrame(data, columns=column_names)
        else:
            df = pd.DataFrame()
        
        # Set pandas display options to show all content
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', None)
        
        # Display the DataFrame
        display(df)
        
        return df
        
    except Exception as e:
        print(f"Error: {e}")
        return None


## Example Tests

Here are some example tests you can run:


In [14]:
# Test castell_statistics with correct parameters
print("Testing castell_statistics with correct parameters:")
df_stats = test_query('concurs_ranking', 
                       #colla=["Moixiganguers d'Igualada"],
                     # castell=['3d9af'],  
                       year=['2018'],
                       jornada = ['Jornada Dissabte Tarragona'],
                      limit=5)


Testing castell_statistics with correct parameters:

TESTING: CONCURS_RANKING
Year: ['2018']
Jornada: ['Jornada Dissabte Tarragona']
Limit: 5
--------------------------------------------------

Results (5 rows):
--------------------------------------------------------------------------------


,edition,title,date,location,plaça,colla_guanyadora,position,colla_name,total_points,jornada,primera_ronda,segona_ronda,tercera_ronda,quarta_ronda,cinquena_ronda,sisena_ronda,setmana_ronda
0,XXVII,XXVIIè Concurs de Castells,30 de setembre de 2018,,"Plaça del Castell, Torredembarra (30 de setembre)Tarraco Arena Plaça, Tarragona (6 i 7 d'octubre)",Colla Vella dels Xiquets de Valls,8,Xiquets de Reus,3460,Jornada Dissabte Tarragona,"{""castell"": ""2d8f"", ""status"": ""Descarregat""}","{""castell"": ""3d9f"", ""status"": ""Descarregat""}","{""castell"": ""5d8"", ""status"": ""Descarregat""}","{""castell"": ""-"", ""status"": """"}","{""castell"": ""-"", ""status"": """"}",None,None
1,XXVII,XXVIIè Concurs de Castells,30 de setembre de 2018,,"Plaça del Castell, Torredembarra (30 de setembre)Tarraco Arena Plaça, Tarragona (6 i 7 d'octubre)",Colla Vella dels Xiquets de Valls,9,Moixiganguers d'Igualada,3460,Jornada Dissabte Tarragona,"{""castell"": ""5d8"", ""status"": ""Descarregat""}","{""castell"": ""3d9f"", ""status"": ""Descarregat""}","{""castell"": ""2d8f"", ""status"": ""Descarregat""}","{""castell"": ""-"", ""status"": """"}","{""castell"": ""-"", ""status"": """"}",None,None
2,XXVII,XXVIIè Concurs de Castells,30 de setembre de 2018,,"Plaça del Castell, Torredembarra (30 de setembre)Tarraco Arena Plaça, Tarragona (6 i 7 d'octubre)",Colla Vella dels Xiquets de Valls,15,Sagals d'Osona,2255,Jornada Dissabte Tarragona,"{""castell"": ""3d8"", ""status"": ""Descarregat""}","{""castell"": ""2d8f"", ""status"": ""Descarregat""}","{""castell"": ""4d8"", ""status"": ""Descarregat""}","{""castell"": ""Pd7f"", ""status"": ""Intent""}","{""castell"": ""-"", ""status"": """"}",None,None
3,XXVII,XXVIIè Concurs de Castells,30 de setembre de 2018,,"Plaça del Castell, Torredembarra (30 de setembre)Tarraco Arena Plaça, Tarragona (6 i 7 d'octubre)",Colla Vella dels Xiquets de Valls,17,Xicots de Vilafranca,1855,Jornada Dissabte Tarragona,"{""castell"": ""3d8"", ""status"": ""Descarregat""}","{""castell"": ""4d9f"", ""status"": ""Intent""}","{""castell"": ""2d7"", ""status"": ""Descarregat""}","{""castell"": ""4d8"", ""status"": ""Carregat""}","{""castell"": ""-"", ""status"": """"}",None,None
4,XXVII,XXVIIè Concurs de Castells,30 de setembre de 2018,,"Plaça del Castell, Torredembarra (30 de setembre)Tarraco Arena Plaça, Tarragona (6 i 7 d'octubre)",Colla Vella dels Xiquets de Valls,18,Colla de Castellers d'Esplugues,1770,Jornada Dissabte Tarragona,"{""castell"": ""2d7"", ""status"": ""Descarregat""}","{""castell"": ""3d8"", ""status"": ""Descarregat""}","{""castell"": ""3d7s"", ""status"": ""Descarregat""}","{""castell"": ""-"", ""status"": """"}","{""castell"": ""-"", ""status"": """"}",None,None


In [ ]:
# # # Test 1: Best diada with multiple castells filter
# # This will find diades where either '5d8' OR '3d8' castells were performed
df1 = test_query('millor_diada', 
                #  colla=['Castellers de Vilafranca'],
                 castell=['3d10fm'],#status='Intent',
                  
                 limit=5)



TESTING: MILLOR_DIADA
Castell: ['3d10fm']
Limit: 5
--------------------------------------------------

Results (5 rows):
--------------------------------------------------------------------------------


,event_id,event_name,event_date,event_place,event_city,colla_name,castells_fets,num_castells,total_punts
0,7359,Concurs de Castells de TarragonaXXVI Concurs de Castells a Tarragona,2016-10-02,Tarraco Arena Plaça,Tarragona,Castellers de Vilafranca,"4d10fm (Descarregat), 3d10fm (Descarregat), 2d8 (Carregat), 2d8 (Carregat)",4,12445
1,7256,Tots Sants a Vilafranca del PenedèsDiada de Tots Sants a Vilafranca del Penedès,2016-11-01,Plaça de la Vila,Vilafranca del Penedès,Castellers de Vilafranca,"3d10fm (Descarregat), 4d9 (Descarregat), 3d9af (Descarregat), Pd8fm (Descarregat), Pd5 (Descarregat), Pd5 (Descarregat), Pd5 (Descarregat)",18,11995
2,875,Concurs de Castells de TarragonaXXIX Concurs de Castells a Tarragona,2024-10-06,Tarraco Arena Plaça,Tarragona,Castellers de Vilafranca,"3d10fm (Descarregat), 9d9f (Carregat), 4d10fm (Carregat), 4d9af (Carregat), 9d9f (Intent desmuntat)",5,11715
3,11238,Sant Fèlix a Vilafranca del PenedèsDiada de Sant Fèlix a Vilafranca del Penedès,2013-08-30,Plaça de la Vila,Vilafranca del Penedès,Castellers de Vilafranca,"3d10fm (Descarregat), 4d9 (Descarregat), 4d9af (Descarregat), Pd8fm (Descarregat), Pd5 (Descarregat), Pd5 (Descarregat)",7,11705
4,6110,Tots Sants a Vilafranca del PenedèsDiada de Tots Sants a Vilafranca del Penedès,2017-11-01,Plaça de la Vila,Vilafranca del Penedès,Castellers de Vilafranca,"3d10fm (Descarregat), 2d8 (Descarregat), 3d9af (Descarregat), Pd8fm (Descarregat), Pd5 (Descarregat), Pd8fm (Intent)",19,11680



TESTING: CASTELL_STATISTICS
Colla: ['Castellers de Vilafranca']
Year: ['3d7']
Limit: 5
--------------------------------------------------
[SQL] Predefined template failed for castell_statistics, falling back to custom LLM generation
Error: llm_call_func is required for custom SQL generation


In [ ]:
df1 = test_query('location_actuations', 
                   colla=['Castellers de Vilafranca'],
                  year=['2024'],
                  limit=5)


TESTING: LOCATION_ACTUATIONS
Colla: ['Castellers de Vilafranca']
Year: ['2024']
Limit: 5
--------------------------------------------------

Results (5 rows):
--------------------------------------------------------------------------------


,event_name,date,place,city,colla_name,num_castells,castells_fets
0,Concurs de Castells de TarragonaXXIX Concurs de Castells a Tarragona,2024-10-06,Tarraco Arena Plaça,Tarragona,Castellers de Vilafranca,5,"3de10fm (Descarregat), 9de9f (Carregat), 4de10fm (Carregat), 4de9fp (Carregat), 9de9f (Intent desmuntat)"
1,Festa Major de l'ArboçFesta Major de l'Arboç,2024-08-25,Plaça de la Vila,L'Arboç,Castellers de Vilafranca,11,"3de10fm (Descarregat), 4de9fp (Descarregat), 5de9f (Descarregat), Pde8fm (Descarregat), Pde4cam (Descarregat)"
2,Festa Major de Vilanova i la GeltrúDiada de les Neus de Vilanova i la Geltrú,2024-08-03,Plaça de la Vila,Vilanova i la Geltrú,Castellers de Vilafranca,10,"3de10fm (Descarregat), 4de9fp (Descarregat), 5de9f (Descarregat), Pde8fm (Descarregat), Pde4cam (Carregat)"
3,Sant Fèlix a Vilafranca del PenedèsDiada de Sant Fèlix a Vilafranca del Penedès,2024-08-30,Plaça de la Vila,Vilafranca del Penedès,Castellers de Vilafranca,6,"3de10fm (Carregat), 4de9fp (Descarregat), Pde8fm (Descarregat), 2de9fm (Carregat), 9de9f (Intent desmuntat), Pde4cam (Descarregat)"
4,Les Santes a MataróFesta Major de Les Santes a Mataró,2024-07-20,Plaça Santa Anna,Mataró,Castellers de Vilafranca,5,"5de9f (Descarregat), Pde8fm (Descarregat), 2de9fm (Descarregat), 9de8 (Descarregat), Pde4cam (Descarregat)"


In [ ]:
# Test castell_statistics with correct parameters
df_stats = test_query('castell_statistics', 
                      colla=['Castellers de Vilafranca'],
                      castell=['3d7'],
                      limit=5)



TESTING: CASTELL_STATISTICS
Colla: ['Castellers de Vilafranca']
Castell: ['3d7']
Limit: 5
--------------------------------------------------
Error: Tipus de consulta SQL no reconegut: castell_statistics


In [ ]:
df1 = test_query('millor_castell', 
                  colla=['Castellers de Vilafranca'],
                  location=['Barcelona'],
                  
                 limit=5)


TESTING: MILLOR_CASTELL
Colla: ['Castellers de Vilafranca']
Location: ['Barcelona']
Limit: 5
--------------------------------------------------

Results (5 rows):
--------------------------------------------------------------------------------


,event_name,date,place,city,colla_name,castell_name,status,punts_descarregat,punts_carregat
0,La Mercè de Barcelona (colles convidades)Diada de la Mercè (colles convidades) a Barcelona,2017-09-23,Plaça de Sant Jaume,Barcelona,Castellers de Vilafranca,4de9,Intent desmuntat,3195,2680
1,La Mercè de Barcelona (colles convidades)Diada de la Mercè (colles convidades) a Barcelona,2014-09-21,Plaça de Sant Jaume,Barcelona,Castellers de Vilafranca,3de9fp,Descarregat,2555,2315
2,La Mercè de Barcelona (colles convidades)Diada de la Mercè (colles convidades) a Barcelona,2013-09-22,Plaça de Sant Jaume,Barcelona,Castellers de Vilafranca,3de9fp,Descarregat,2555,2315
3,La Mercè de Barcelona (colles convidades)Diada de la Mercè (colles convidades) a Barcelona,2011-09-25,Plaça Sant Jaume,Barcelona,Castellers de Vilafranca,3de9fp,Descarregat,2555,2315
4,La Mercè de Barcelona (colles convidades)Diada de la Mercè (colles convidades) a Barcelona,2025-09-28,Plaça Sant Jaume,Barcelona,Castellers de Vilafranca,5de9f,Descarregat,2400,2090
